https://github.com/philipperemy/timit

https://www.kaggle.com/datasets/mfekadu/darpa-timit-acousticphonetic-continuous-speech?resource=download

https://www.kaggle.com/code/julwan/phoneme-recognition-with-wav2vec2

wav2vec seems like a common transformer model for speech recognition. 

630 speakers, each read 10 sentences—2 dialect "shibboleth" sentences (SA), 5 phonetically compact sentences (SX), and 3 phonetically diverse sentences (SI).

- SA: expose the dialectal variants of the speakers.
- SX: provide a good coverage of pairs of phones, with extra occurrences of phonetic contexts thought to be either difficult or of particular interest.
- SI: add diversity in sentence types and phonetic contexts.



| Sentence Type | #Sentences | #Speakers | Total | #Sentences/Speaker |
|---------------|------------|-----------|-------|--------------------|
| Dialect (SA)  |      2     |    630    | 1260  |         2          |
| Compact (SX)  |     450    |     7     | 3150  |         5          |
| Diverse (SI)  |    1890    |     1     | 1890  |         3          |
|---------------|------------|-----------|-------|--------------------|
| Total         |    2342    |           | 6300  |        10          |


| Dialect Region (dr) | #Male   | #Female  | Total   |
|----------------------|---------|----------|---------|
|         1            | 31 (63%)| 18 (27%) | 49 (8%) |
|         2            | 71 (70%)| 31 (30%) |102(16%)|
|         3            | 79 (67%)| 23 (23%) |102(16%)|
|         4            | 69 (69%)| 31 (31%) |100(16%)|
|         5            | 62 (63%)| 36 (37%) | 98(16%)|
|         6            | 30 (65%)| 16 (35%) | 46 (7%)|
|         7            | 74 (74%)| 26 (26%) |100(16%)|
|         8            | 22 (67%)| 11 (33%) | 33 (5%)|
|----------------------|---------|----------|---------|
|        Total         | 438(70%)|192(30%)  |630(100%)|

The dialect regions are:
   dr1: New England
   dr2: Northern
   dr3: North Midland
   dr4: South Midland
   dr5: Southern
   dr6: New York City
   dr7: Western
   dr8: Army Brat (moved around)


For each utterance, the following information is available:
- .wav file: the speech waveform.
- .phn file: Time-aligned phonetic transcription.
- .wrd file: Time-aligned word transcription.
- .txt file: Associated orthographic transcription of the words the
            person said.

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import glob
import pandas as pd

# custum modules
from mfcc_label import *
from utils import *
from config import TEST_PHN_PATH
from get_prob import *

## Load phoneme, word, and transcript data

In [6]:
# sx61
glob.glob('timit/data/TRAIN/DR4/MGAG0/SX61.*')

['timit/data/TRAIN/DR4/MGAG0/SX61.TXT',
 'timit/data/TRAIN/DR4/MGAG0/SX61.WAV',
 'timit/data/TRAIN/DR4/MGAG0/SX61.PHN',
 'timit/data/TRAIN/DR4/MGAG0/SX61.WRD',
 'timit/data/TRAIN/DR4/MGAG0/SX61.WAV.wav']

In [7]:
# phoneme
df_phoneme = load_data(glob.glob('timit/data/TRAIN/DR4/MGAG0/SX61.PHN')[0], "phoneme")
df_phoneme.tail()

,start_sample,end_sample,phoneme,diff_sample
32,36940,39000,#b,2060
33,39000,40200,#b,1200
34,40200,40960,#b,760
35,40960,41720,#b,760
36,41720,43680,h#,1960


In [ ]:
# word
df_word = load_data(glob.glob('timit/data/TRAIN/DR4/MGAG0/SX61.WRD')[0], "word")
df_word.tail()

In [ ]:
df_transcript = load_transcript('timit/data/TRAIN/DR4/MGAG0/SX61.TXT')
df_transcript.tail()

## MFCC
Note that we can always use smaller `win_length` and `hop_length` for more fine-grained alignment.

if we choose 25ms window and 5ms hop, we can have $25 * 10^-3 * 16000 = 400$ `win_length` and $5 * 10^-3 * 16000 = 80$ `hop_length`.

https://librosa.org/doc/main/generated/librosa.feature.mfcc.html#librosa.feature.mfcc

In [8]:
df_mfcc = process_audio_file('timit/data/TRAIN/DR4/MGAG0/SX61.WAV',
                             win_length=400, 
                             hop_length=80)
display(df_mfcc.tail())

print(df_mfcc.shape)

,start_sample,end_sample,mfcc
541,43280,43680,"[-827.1031, 32.212986, 10.357775, 4.273916, -7..."
542,43360,43760,"[-833.8036, 18.833408, 6.527279, 11.398876, 0...."
543,43440,43840,"[-845.2821, 3.8058267, 1.8199315, 11.270794, 5..."
544,43520,43920,"[-847.30914, 4.7726836, 6.2668734, 6.074581, 8..."
545,43600,44000,"[-847.56964, 9.320388, 15.307094, 7.9442987, 1..."


(546, 3)


### draft/explanation

y: total number of samples

sr: number of samples per second

In [ ]:
y, sr = librosa.load('timit/data/TRAIN/DR4/MGAG0/SX61.WAV', sr=16000)
y.shape, sr

In [ ]:
y, sr = librosa.load('timit/data/TRAIN/DR4/MGAG0/SX61.WAV')
y.shape, sr

In [ ]:
# duration
60259/22050

we know y=43725

In [ ]:
load_transcript('timit/data/TRAIN/DR4/MGAG0/SX61.TXT')

In [ ]:
# try another 
y, sr = librosa.load('timit/data/TRAIN/DR4/MGAG0/SA1.WAV', sr=16000)
y.shape, sr

In [ ]:
load_transcript('timit/data/TRAIN/DR4/MGAG0/SA1.TXT')

In [ ]:
# Number of samples=Duration×Sampling frequency (Sr in Hz)
# Hz represents the number of samples per second.
duration = len(y) / sr
duration

In [ ]:
43725/duration  
# if an audio file is 2.732834467120181 seconds long and contains 43725 samples, it means that the audio file has a sampling rate of approximately 16000 Hz

In [ ]:
n_mfcc = 20  # Number of MFCC coefficients
hop_length = 512  # Number of samples between consecutive frames (frame step)
win_length = 1024  # Length of the analysis window in samples (frame size)

In [ ]:
1024/16000  # 64 ms

In [ ]:
# Extract MFCC features
mfccs = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, win_length=win_length, n_mfcc=n_mfcc)

In [ ]:
mfccs.shape
# The first dimension (20) represents the number of MFCC coefficients extracted. This is a common default value, as 20 coefficients are often sufficient to capture relevant information about the spectral characteristics of the audio signal.
# The second dimension (86) represents the number of frames extracted from the audio signal. The number of frames is determined by the duration of the audio signal and the frame length and hop length used in the feature extraction process.

## Align df_mfcc and df_phoneme 

epi, pau, h# are silence.

In [11]:
df_phoneme = load_data(TEST_PHN_PATH, "phoneme")
df_phoneme = split_phoneme(df_phoneme)
display(df_phoneme[(df_phoneme['start_sample']>=16000) & (df_phoneme['end_sample']<=21000)] )

,start_sample,end_sample,phoneme,diff_sample
13,16510.000000,18046.000000,#b,1536
14,18046.000000,18680.000000,#b,634
15,18680.000000,19173.333333,b-axr,1480
16,19173.333333,19666.666667,m-axr,1480
17,19666.666667,20160.000000,e-axr,1480


In [ ]:
# Adjust display settings 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
new_df_mfcc = label_df_mfcc(df_mfcc, df_phoneme) 
new_df_mfcc[(df_mfcc['start_sample']>=16000) & (df_mfcc['end_sample']<=21000)] 

In [ ]:
new_df_mfcc2 = vectorize_label_df_mfcc(new_df_mfcc, df_phoneme)
new_df_mfcc2[(new_df_mfcc2['start_sample']>=16000) & (new_df_mfcc2['end_sample']<=21000)] 

In [14]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

## Prepare DNN training set

In [1]:
from mfcc_label import main

In [ ]:
main()